In [1]:
import pandas as pd
import numpy as np
import openpyxl
import re
from re import sub
from decimal import Decimal
import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from numpy import unique, where
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.metrics import r2_score, silhouette_samples, silhouette_score , completeness_score , homogeneity_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from IPython.display import display, HTML, display_html
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer


/Users/chenyangkang/miniforge3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
df = pd.read_csv('../../Data/LA_Airbnb/listings_detailed.csv')

In [4]:
prohibitedWords = ['private', 'shared', ' in ', 'entire', 'room'] # added spaces in front and behind 'in' to ensure accuracy
big_regex = re.compile('|'.join(map(re.escape, prohibitedWords)))
df['property_type_clean'] =  df['property_type'].apply(lambda x: big_regex.sub("", x).strip())
df['property_type_clean'] = np.where(df['property_type_clean']=='',df['room_type'],df['property_type_clean'])


In [52]:
from collections import Counter
a=Counter()
a.update(df['property_type_clean'].values)
sorted_property_types = sorted(a.items(), key=lambda x:-x[1])
all_sum = np.nansum([i[1] for i in sorted_property_types])
sorted_property_types = [(a,float(b)/all_sum) for a,b in sorted_property_types]
pickle.dump(sorted_property_types, open(f'./utiles/sorted_property_types.pkl','wb'))


In [57]:
a=Counter()
a.update(df['room_type'].values)
sorted_room_types = sorted(a.items(), key=lambda x:-x[1])
all_sum = np.nansum([i[1] for i in sorted_room_types])
sorted_room_types = [(a,float(b)/all_sum) for a,b in sorted_room_types]
pickle.dump(sorted_room_types, open(f'./utiles/sorted_room_types.pkl','wb'))

In [58]:
a_u = open(r"./utiles/amenities_universe.txt", "r",encoding='unicode escape').read().split('\n')
bb = pd.read_csv('./final_features/LA_extracted_all_features_imputed.csv')
bb = bb[[i for i in bb.columns if 'Amenities' in i]]
bb.columns = [i.split('Amenities_')[1] for i in bb.columns]
bb = sorted(
    dict(bb[[i for i in bb.columns if i in a_u]].sum(axis=0)).items(),key=lambda x:-x[1]
)
all_sum = np.nansum([i[1] for i in bb])
bb = [(a,float(b)/all_sum) for a,b in bb]
pickle.dump(bb, open(f'./utiles/sorted_amenities.pkl','wb'))
